In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from plotting import *
from regression import *
from utils import *

In [3]:
# Load two results CSVs
results_name1 = 'hardware-capex-energy-top_n=10-original'
results_name2 = 'cloud-top_n=10-original'
results_dir1 = f'results/{results_name1}/'
results_dir2 = f'results/{results_name2}/'
results_dir = f'results/{results_name1}-vs-{results_name2}/'
os.makedirs(results_dir, exist_ok=True)

In [4]:
cost_df1 = pd.read_csv(results_dir1 + 'cost_dataset.csv')
cost_df2 = pd.read_csv(results_dir2 + 'cost_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'results/cloud-top_n=10-original/cost_dataset.csv'

In [ ]:
col = 'Cost (inflation-adjusted)'

In [ ]:
# Relative error stats
ratios = cost_df2[col] / cost_df1[col]
print(f'Ratio stats: {results_name2} / {results_name1}')
print_median_and_ci(ratios.dropna(), ci=[2.5, 97.5])
print(f'Mean: {ratios.mean():.2g}')

Ratio stats: old-May-26/hardware-capex-energy-top_n=10-original / hardware-capex-energy-top_n=10-original
Median: 1.1 [95% CI: 0.04, 37]
Mean: 4.8


In [ ]:
len(ratios.dropna())

32

In [ ]:
cost_df1[cost_df1['System'] == 'Inflection-2'][col] / cost_df2[cost_df2['System'] == 'Inflection-2'][col]

1    0.999992
Name: Cost (inflation-adjusted), dtype: float64

In [ ]:
cost_df1.iloc[ratios.argmin()][col], cost_df2.iloc[ratios.argmin()][col]

(471445.3247973037, 3863.7677655638927)

In [ ]:
cost_df1.iloc[ratios.argmax()][col], cost_df2.iloc[ratios.argmax()][col]

(92453.37635669748, 3704322.4296740578)

# Difference in regression

In [ ]:
cost_df1['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df1['Publication date']))
cost_df2['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df2['Publication date']))

In [ ]:
# Compare slopes
regression_slope_t_test(cost_df1, cost_df2, ['Publication date (float)'], col, logy=True)

Slope 1: 0.40 (SE: 0.05)
Slope 2: 0.37 (SE: 0.05)
Correlation between methods: 1.00
Test statistic: 5.27
p-value: 0.00


In [ ]:
# Compare slopes and intercepts
chow_test(cost_df1, cost_df2, ['Publication date (float)'], col, logy=True)

Chow Test F-statistic: 0.07701927041535685
p-value: 0.9259336530332253


(0.07701927041535685, 0.9259336530332253)

In [ ]:
set(cost_df1.dropna(subset=['Cost'])['System']) - set(cost_df2.dropna(subset=['Cost'])['System']) 

{'Flan-PaLM 540B',
 'Gemini 1.0 Ultra',
 'HyperCLOVA 82B',
 'Transformer (Adaptive Input Embeddings) WT103'}

In [ ]:
set(cost_df2.dropna(subset=['Cost'])['System']) - set(cost_df1.dropna(subset=['Cost'])['System']) 

{'Gemini Ultra', 'HyperCLOVA', 'LLaMA-65B', 'Llama 2-70B', 'xTrimoPGLM -100B'}

In [ ]:
cost_df1[cost_df1['System'] == 'Flan-PaLM 540B']

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted),Publication date (float),Source,Size
10,Flan-PaLM 540B,Language,Language modelling/generation,Unreleased,Scaling Instruction-Finetuned Language Models,2022-10-20,Google,5.400000e+11,2.500000e+24,NaN,...,5.0,512.0,0.3,NaN,NaN,2.903292e+06,2.961791e+06,2022.802055,hardware-capex-energy-top_n=10-original,0.5


# Plots

In [ ]:
# Add a new column to differentiate the DataFrames
cost_df1['Source'] = results_name1
cost_df2['Source'] = results_name2

# Combine the DataFrames
combined_cost_df = pd.concat([cost_df1, cost_df2])

In [ ]:
fig = px.scatter(
    combined_cost_df,
    x='Publication date',
    y=col,
    color='Source',
    hover_data=['System'],
    log_y=True,
)
fig.update_traces(textposition='top center')

# legend
fig.update_layout(
    legend_title_text='',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=0.05,
        xanchor='right',
        x=0.95,
    ),
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter_comparison')

fig.show()